<a href="https://colab.research.google.com/github/u6yuvi/NER-System/blob/main/notebooks/NER_Fientune_Distilbert_conll2002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference
https://github.com/huggingface/notebooks/blob/main/examples/token_classification.ipynb

# Imports

In [ ]:
!pip install datasets transformers seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 99.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 18.0 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16179 sha256=140fd167454da47f4dba1c3e088b061c8daf7af753a8e16950bd991a4bc3cce1
  Stored in directory: /root/.cache/pip/wheels/ad/5c/ba/05fa33fa5

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [ ]:
if IN_COLAB:
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import json

In [ ]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "distilbert-base-cased"#"bert-base-cased"
batch_size = 32

# Conll2002 Data Set

In [ ]:
from datasets import load_dataset
dataset = load_dataset("conll2002", "es")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/8324 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1916 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1518 [00:00<?, ? examples/s]

Dataset conll2002 downloaded and prepared to /root/.cache/huggingface/datasets/conll2002/es/1.0.0/a3a8a8612caf57271f5b35c5ae1dd25f99ddb9efb9c1667abaa70ede33e863e5. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'ner_tags'],
        num_rows: 8324
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'ner_tags'],
        num_rows: 1916
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'ner_tags'],
        num_rows: 1518
    })
})

In [ ]:
dataset["train"][0]

{'id': '0',
 'tokens': ['Melbourne',
  '(',
  'Australia',
  ')',
  ',',
  '25',
  'may',
  '(',
  'EFE',
  ')',
  '.'],
 'pos_tags': [29, 21, 29, 22, 13, 59, 28, 21, 28, 22, 20],
 'ner_tags': [5, 0, 5, 0, 0, 0, 0, 0, 3, 0, 0]}

In [ ]:
dataset["train"].features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'pos_tags': Sequence(feature=ClassLabel(names=['AO', 'AQ', 'CC', 'CS', 'DA', 'DE', 'DD', 'DI', 'DN', 'DP', 'DT', 'Faa', 'Fat', 'Fc', 'Fd', 'Fe', 'Fg', 'Fh', 'Fia', 'Fit', 'Fp', 'Fpa', 'Fpt', 'Fs', 'Ft', 'Fx', 'Fz', 'I', 'NC', 'NP', 'P0', 'PD', 'PI', 'PN', 'PP', 'PR', 'PT', 'PX', 'RG', 'RN', 'SP', 'VAI', 'VAM', 'VAN', 'VAP', 'VAS', 'VMG', 'VMI', 'VMM', 'VMN', 'VMP', 'VMS', 'VSG', 'VSI', 'VSM', 'VSN', 'VSP', 'VSS', 'Y', 'Z'], id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)}

In [ ]:
label2ids = {j:i for i,j in enumerate(dataset["train"].features['ner_tags'].feature.names)}
label2ids

{'O': 0,
 'B-PER': 1,
 'I-PER': 2,
 'B-ORG': 3,
 'I-ORG': 4,
 'B-LOC': 5,
 'I-LOC': 6,
 'B-MISC': 7,
 'I-MISC': 8}

In [ ]:
id2labels = {i:j for j,i in label2ids.items()}
id2labels

{0: 'O',
 1: 'B-PER',
 2: 'I-PER',
 3: 'B-ORG',
 4: 'I-ORG',
 5: 'B-LOC',
 6: 'I-LOC',
 7: 'B-MISC',
 8: 'I-MISC'}

In [ ]:
label_list = list(label2ids.keys())

In [ ]:
from collections import Counter, defaultdict

split2freqs = defaultdict(Counter) 
for data in dataset["train"]:
  for tag in data["ner_tags"]: 
      # if tag.startswith("B"):
      #   tag_type = tag.split("-")[1] 
        split2freqs["train"][tag] += 1

pd.DataFrame.from_dict(split2freqs, orient="index")

,5,0,3,1,2,7,4,6,8
train,4913,231920,7390,4321,3903,2173,4992,1891,3212


In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
label_all_tokens = True
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True,padding=True)
    #print(len(tokenized_input))
    labels = []
    #print("example",examples["ner_tag"])
    #print(len(examples["ner_tag"]))
    for i, label in enumerate(examples["ner_tags"]):
        #print(tokenizer.convert_ids_to_tokens(tokenized_inputs["input_ids"][i]))
        #print("i-label",i,label)
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        #print("word_ids",i,word_ids)
        #print("ner_tag",examples["ner_tag"][i])
        #print(i,label)
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)
        #print("labels",label)

    #print(labels)
    tokenized_inputs["labels"] = labels
    #print(tokenized_inputs)
    return tokenized_inputs

In [ ]:
train_tokenized_datasets = dataset["train"].map(tokenize_and_align_labels,batched=True,batch_size=512)

  0%|          | 0/17 [00:00<?, ?ba/s]

In [ ]:
validation_tokenized_datasets = dataset["validation"].map(tokenize_and_align_labels,batched=True,batch_size=512)

  0%|          | 0/4 [00:00<?, ?ba/s]

In [ ]:
test_tokenized_datasets = dataset["test"].map(tokenize_and_align_labels,batched=True,batch_size=512)

  0%|          | 0/3 [00:00<?, ?ba/s]

In [ ]:
len(dataset["train"]),len(dataset["validation"]),len(dataset["test"])

(8324, 1916, 1518)

In [ ]:
# remove_cols = ['hash_id', 'doc_type', 'file_name', 'text', 'tokens', 'wf_id', 'bio_tag', 'toks_mask', 'toks_mask_perct',\
#                'line_id', 'page_id', 'tags', 'token_tags',\
#                'tok_len', 'tag_len', 'ner_tag', '__index_level_0__']

In [ ]:
# tokenized_datasets = tokenized_datasets.remove_columns(remove_cols)

In [ ]:
# tokenized_datasets

# Setting up Huggingface Trainer

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label2ids),id2label = id2labels,
                                                        label2id =label2ids )

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this 

In [ ]:
#??TrainingArguments

In [ ]:
#IntervalStrategy.STEPS
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments("/content/drive/MyDrive/Analytics/models/exp1/"+f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    eval_steps=100,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    push_to_hub=False,
    #label_names = label_list,
    save_total_limit = 3,
    do_eval=True,
    load_best_model_at_end = True,
    metric_for_best_model='eval_loss',#use_mps_device=True
    
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
from datasets import load_dataset, load_metric
metric = load_metric("seqeval")

<ipython-input-27-498c34c2054a>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    #print(predictions)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }
    return results

In [ ]:
from transformers import EarlyStoppingCallback

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=validation_tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Train Model

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: ner_tags, tokens, id, pos_tags. If ner_tags, tokens, id, pos_tags are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 8324
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 261
  Number of trainable parameters = 65197833


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.157825,0.730948,0.739374,0.735137,0.955908


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: ner_tags, tokens, id, pos_tags. If ner_tags, tokens, id, pos_tags are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1916
  Batch size = 32
Saving model checkpoint to /content/drive/MyDrive/Analytics/models/exp1/distilbert-base-cased-finetuned-ner/checkpoint-261
Configuration saved in /content/drive/MyDrive/Analytics/models/exp1/distilbert-base-cased-finetuned-ner/checkpoint-261/config.json
Model weights saved in /content/drive/MyDrive/Analytics/models/exp1/distilbert-base-cased-finetuned-ner/checkpoint-261/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Analytics/models/exp1/distilbert-base-cased-finetuned-ner/checkpoint-261/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Analytics/models/exp1

TrainOutput(global_step=261, training_loss=0.06975474302796111, metrics={'train_runtime': 388.6441, 'train_samples_per_second': 21.418, 'train_steps_per_second': 0.672, 'total_flos': 1078089836834304.0, 'train_loss': 0.06975474302796111, 'epoch': 1.0})

In [ ]:
#active_logits = output.logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
# flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
# # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
# active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
# targets = torch.masked_select(flattened_targets, active_accuracy)
# predictions = torch.masked_select(flattened_predictions, active_accuracy)

In [ ]:
# import torch
# torch.argmax(active_logits,axis=1)

In [ ]:

trainer.evaluate()

***** Running Evaluation *****
  Num examples = 51753
  Batch size = 64


{'eval_loss': 0.02408629097044468,
 'eval_precision': 0.9843872280482079,
 'eval_recall': 0.9852289416141771,
 'eval_f1': 0.9848079049784532,
 'eval_accuracy': 0.9912822210537932,
 'eval_runtime': 35.3256,
 'eval_samples_per_second': 1465.027,
 'eval_steps_per_second': 22.901,
 'epoch': 20.0}

In [ ]:
#no-dups
predictions, labels, _ = trainer.predict(tokenized_datasets)
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

***** Running Prediction *****
  Num examples = 51753
  Batch size = 64


{'CARDINAL': {'precision': 0.9949883463958361,
  'recall': 0.9966372323696625,
  'f1': 0.9958121068184702,
  'number': 82670},
 'COMMODITY': {'precision': 0.9724862431215607,
  'recall': 0.9571639586410635,
  'f1': 0.9647642679900744,
  'number': 2031},
 'DATE': {'precision': 0.9824980670555985,
  'recall': 0.9862414450010584,
  'f1': 0.9843661971830986,
  'number': 14173},
 'EMAIL': {'precision': 0.9973208305425318,
  'recall': 0.994988306047444,
  'f1': 0.9961532028767353,
  'number': 2993},
 'LOC': {'precision': 0.9713128306878307,
  'recall': 0.9698695723955754,
  'f1': 0.9705906650144568,
  'number': 12114},
 'ORG': {'precision': 0.965421124001808,
  'recall': 0.9603567146282974,
  'f1': 0.9628822601247277,
  'number': 26688},
 'QUANTITY_TYPE': {'precision': 0.9405436013590034,
  'recall': 0.9730521382542472,
  'f1': 0.9565217391304348,
  'number': 1707},
 'UNIT': {'precision': 0.9706707830479933,
  'recall': 0.9803004535147393,
  'f1': 0.975461853053166,
  'number': 7056},
 'URL'

# Inference on Finetuned Model

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Analytics/models/exp1/distilbert-base-cased-finetuned-ner/checkpoint-261")

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained("/content/drive/MyDrive/Analytics/models/exp1/distilbert-base-cased-finetuned-ner/checkpoint-261", num_labels=len(label2ids),id2label = id2labels,
                                                        label2id =label2ids )

In [ ]:
model_name = "/content/drive/MyDrive/Analytics/models/exp1/distilbert-base-cased-finetuned-ner/checkpoint-261"
task = 'ner'
batch_size = 64
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    eval_steps=100,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    push_to_hub=False,
    #label_names = label_list,
    save_total_limit = 3,
    do_eval=True,
    do_train=False,
    load_best_model_at_end = True,
    metric_for_best_model='eval_loss'
    
)

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=test_tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    #compute_metrics=compute_metrics
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

In [ ]:
#with-dups
predictions, labels, _ = trainer.predict(test_tokenized_datasets)
predictions = np.argmax(predictions, axis=2)

# # Remove ignored index (special tokens)
# true_predictions = [
#     [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
#     for prediction, label in zip(predictions, labels)
# ]
# true_labels = [
#     [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
#     for prediction, label in zip(predictions, labels)
# ]

# results = metric.compute(predictions=true_predictions, references=true_labels)
# results

The following columns in the test set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: pos_tags, id, ner_tags, tokens. If pos_tags, id, ner_tags, tokens are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1518
  Batch size = 64


In [ ]:
len(predictions)

1518

In [ ]:
# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'LOC': {'precision': 0.8014268727705113,
  'recall': 0.770579268292683,
  'f1': 0.785700408004663,
  'number': 2624},
 'MISC': {'precision': 0.5101351351351351,
  'recall': 0.5112866817155757,
  'f1': 0.5107102593010147,
  'number': 886},
 'ORG': {'precision': 0.7910740559097597,
  'recall': 0.8568393094289508,
  'f1': 0.8226443962769349,
  'number': 3765},
 'PER': {'precision': 0.8219645293315143,
  'recall': 0.8776402039329935,
  'f1': 0.848890454385347,
  'number': 1373},
 'overall_precision': 0.7711892797319932,
 'overall_recall': 0.7985661424606846,
 'overall_f1': 0.7846389819917059,
 'overall_accuracy': 0.9703692688971499}

In [ ]:
!git clone https://github.com/davidsbatista/NER-Evaluation.git

Cloning into 'NER-Evaluation'...
remote: Enumerating objects: 234, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 234 (delta 0), reused 2 (delta 0), pack-reused 230
Receiving objects: 100% (234/234), 82.22 KiB | 4.57 MiB/s, done.
Resolving deltas: 100% (117/117), done.


In [ ]:
%cd NER-Evaluation
from ner_evaluation.ner_eval import collect_named_entities
from ner_evaluation.ner_eval import compute_metrics
from ner_evaluation.ner_eval import compute_precision_recall_wrapper

/content/NER-Evaluation


In [ ]:
#true_labels

In [ ]:
label_names = set(["".join(i.split("-")[1:]) for i in list(label2ids.keys()) if i!= "O"])

In [ ]:
label_names

{'LOC', 'MISC', 'ORG', 'PER'}

In [ ]:
from ner_evaluation.ner_eval import Evaluator
evaluator = Evaluator(true_labels, true_predictions, label_names)
results, results_agg = evaluator.evaluate()

In [ ]:
results

{'ent_type': {'correct': 7090,
  'incorrect': 1186,
  'partial': 0,
  'missed': 431,
  'spurious': 679,
  'possible': 8707,
  'actual': 8955,
  'precision': 0.7917364600781687,
  'recall': 0.8142873550017228},
 'partial': {'correct': 8005,
  'incorrect': 0,
  'partial': 271,
  'missed': 431,
  'spurious': 679,
  'possible': 8707,
  'actual': 8955,
  'precision': 0.9090452261306533,
  'recall': 0.934937406684277},
 'strict': {'correct': 6906,
  'incorrect': 1370,
  'partial': 0,
  'missed': 431,
  'spurious': 679,
  'possible': 8707,
  'actual': 8955,
  'precision': 0.7711892797319932,
  'recall': 0.7931549328126795},
 'exact': {'correct': 8005,
  'incorrect': 271,
  'partial': 0,
  'missed': 431,
  'spurious': 679,
  'possible': 8707,
  'actual': 8955,
  'precision': 0.8939140145170296,
  'recall': 0.9193752153439761}}